In [1]:
import sys
import struct
import argparse

import numpy as np
import openhd as hd

/home/hpc/iwi3/iwi3083h/.local/lib/python3.8/site-packages/skcuda/cublas.py:284: UserWarning: creating CUBLAS context to get version number
  warnings.warn('creating CUBLAS context to get version number')


In [2]:
Q = 10
D = 10000
hd.init(D=D, context=globals())


In [3]:
train_filename = "/home/hpc/iwi3/iwi3083h/openhd/examples/dataset/isolet_train.choir_dat"
test_filename = "/home/hpc/iwi3/iwi3083h/openhd/examples/dataset/isolet_test.choir_dat"

In [4]:
feature_matrix, labels, n_classes = hd.utils.read_choir_dat(train_filename)

In [5]:
labels

array([ 0,  0,  1, ..., 23, 24, 25], dtype=int32)

In [6]:
feature_matrix

array([[-0.4394, -0.093 ,  0.1718, ...,  0.641 ,  0.5898, -0.4872],
       [-0.4348, -0.1198,  0.2474, ...,  0.4318,  0.4546, -0.091 ],
       [-0.233 ,  0.2124,  0.5014, ...,  0.254 ,  0.1588, -0.4762],
       ...,
       [-0.5824, -0.1646,  0.1406, ...,  0.6812,  0.517 ,  0.343 ],
       [ 0.016 ,  0.8168,  1.    , ...,  0.1034, -0.1954, -0.862 ],
       [-0.6116, -0.104 ,  0.2566, ..., -0.0536,  0.0714, -0.0892]],
      dtype=float32)

In [7]:
feature_matrix_tst, labels_tst, n_classes_tst = hd.utils.read_choir_dat(test_filename)

feature_matrix, feature_matrix_tst = \
        hd.utils.MatrixNormalizer().norm_two(
                feature_matrix, feature_matrix_tst)
N_tst = feature_matrix_tst.shape[0]

In [8]:
feature_matrix.size * feature_matrix.itemsize / (1024*1024) # MB

14.682182312011719

In [9]:
feature_matrix_tst.size * feature_matrix_tst.itemsize / (1024*1024) # MB

3.6693687438964844

In [10]:
N = feature_matrix.shape[0]
F = feature_matrix.shape[1]
N, F

(6238, 617)

In [13]:
# Allocated Memory

(N * D * np.dtype(np.float32).itemsize) / (1024*1024*1024)

0.2323836088180542

In [14]:
@hd.run
def create_random_bases():
    id_base = hd.draw_random_hypervector()
    level_base = hd.draw_random_hypervector()
    return id_base, level_base


@hd.run
def create_ids(F, id_base):
    id_hvs = hd.hypermatrix(F) # np.zeros(F, N) (not the empty list) 
    for f in range(F):
        id_hvs[f] = hd.permute(id_base, f)

    return id_hvs

@hd.run
def create_levels(Q, level_base):
    level_hvs = hd.hypermatrix(Q+1) # np.zeros((Q+1), N) (not the empty list)
    for q in range(Q+1):
        idx = int(q/float(Q) * D) / 2
        level_hvs[q] = hd.flip(level_base, idx)
        level_hvs[q] = hd.shuffle(level_hvs[q], 0)

    return level_hvs

In [15]:
with hd.utils.timing("Base hypervectors"):
    id_base, level_base = create_random_bases()
    id_hvs = create_ids(F, id_base)
    level_hvs = create_levels(Q, level_base)


Base hypervectors	0.01959848403930664


In [16]:
def preprocesser(
        org_feature, cnv_feature, # Predefined argument (single feature)
        Q, level_hvs, id_hvs): # arguments passed by args
    cnv_feature = int(org_feature * Q)


def encoder(
        input_features, output_hypervector, # Predefined arguments
        Q, level_hvs, id_hvs): # arguments passed by args
    for f in range(F):
        output_hypervector += level_hvs[input_features[f]] * id_hvs[f]

In [17]:
with hd.utils.timing("Encode training"):
    hv_matrix = hd.encode(
            encoder, extra_args = (Q, level_hvs, id_hvs),
            feature_matrix = feature_matrix,
            preprocess_function = preprocesser # optional
            )

[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [output_hypervector___base_n_____n___2964e176 : __ARG__output_hypervector]
  <class '_ast.For'> :  <RPT: F>
    <class '_ast.Assign'> :  [output_hypervector___base_n_____n___2964e176 : __ARG__id_hvs*__ARG__level_hvs + output_hypervector___base_n_____n___2964e176]
  <class '_ast.Assign'> :  [__ARG__output_hypervector : output_hypervector___base_n_____n___2964e176]

{'__n__': <class 'int'>, '__blockIdx_y__': <class 'int'>, '__base_n__': 'int', '__N__': <class 'int'>, '__blockDim_x__': <class 'int'>, '__F__': <class 'int'>, '__threadIdx_x__': <class 'int'>, 'F_PER_THREAD': <class 'int'>, 'sample_idx_in_stream': <class 'int'>, '__stream__': 'int', '__M__': <class 'int'>, '__f__': <class 'int'>, '__f_idx__': <class 'int'>, 'org_feature': <class 'float'>, 'input_features': 'np_float_array_type', 'cnv_feature': <class 'int'>, 'Q': <class 'int'>, '__shared_features__': 'np_float_array_type', '__d__': <class 'int

In [18]:
with hd.utils.timing("Encode testing"):
    hv_matrix_tst = hd.encode(
            encoder, extra_args = (Q, level_hvs, id_hvs),
            feature_matrix = feature_matrix_tst,
            preprocess_function = preprocesser # optional
            )


[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [output_hypervector___base_n_____n___2964e176 : __ARG__output_hypervector]
  <class '_ast.For'> :  <RPT: F>
    <class '_ast.Assign'> :  [output_hypervector___base_n_____n___2964e176 : __ARG__id_hvs*__ARG__level_hvs + output_hypervector___base_n_____n___2964e176]
  <class '_ast.Assign'> :  [__ARG__output_hypervector : output_hypervector___base_n_____n___2964e176]

{'__n__': <class 'int'>, '__blockIdx_y__': <class 'int'>, '__base_n__': 'int', '__N__': <class 'int'>, '__blockDim_x__': <class 'int'>, '__F__': <class 'int'>, '__threadIdx_x__': <class 'int'>, 'F_PER_THREAD': <class 'int'>, 'sample_idx_in_stream': <class 'int'>, '__stream__': 'int', '__M__': <class 'int'>, '__f__': <class 'int'>, '__f_idx__': <class 'int'>, 'org_feature': <class 'float'>, 'input_features': 'np_float_array_type', 'cnv_feature': <class 'int'>, 'Q': <class 'int'>, '__shared_features__': 'np_float_array_type', '__d__': <class 'int

## Testing the size for which encoding breaks

In [ ]:
worked = []
for i in range(feature_matrix_tst.shape[1], 0, -1):
    print(i)
    cropped_matrix = feature_matrix_tst[:2, :i]
    worked.append(cropped_matrix.size * cropped_matrix.itemsize / (1024))# KB
    try:
        hv_matrix_tst = hd.encode(
            encoder, extra_args = (Q, level_hvs, id_hvs),
            feature_matrix = cropped_matrix,
            preprocess_function = preprocesser # optional
        )
    except:
        break
    

In [ ]:
np.array(worked )*1024

In [ ]:
@hd.run
def single_pass(hv_matrix, labels, N, n_classes):
    class_hvs = hd.hypermatrix(n_classes)

    for idx in range(N):
        class_hvs[labels[idx]] += hv_matrix[idx]

    return class_hvs

In [ ]:
with hd.utils.timing("Single pass"):
    class_hvs = single_pass(hv_matrix, labels, N, n_classes)
    class_hvs.debug_print_values()

In [ ]:
def validate(labels, pred_labels):
    n_correct = (pred_labels == labels).sum()
    n_labels = len(labels)
    print(n_correct, n_labels, n_correct / float(n_labels) * 100)


In [ ]:
@hd.run
def retrain(class_hvs, hv_matrix, labels, N, n_classes):
    search_results = hd.search(class_hvs, hv_matrix)

    for idx in range(N):
        if search_results[idx] != labels[idx]:
            class_hvs[labels[idx]] += hv_matrix[idx]
            class_hvs[search_results[idx]] -= hv_matrix[idx]

    return class_hvs


In [ ]:
RETRAIN_ITERATIONS = 100
SHOW_STEP_RESULT = True
for it in range(RETRAIN_ITERATIONS):
    with hd.utils.timing("Retrain itereation: %d" % it):
        class_hvs = retrain(class_hvs, hv_matrix, labels, N, n_classes)

    if SHOW_STEP_RESULT and labels_tst is not None:
        validate(labels_tst, hd.search(class_hvs, hv_matrix_tst).to_numpy())

In [ ]:
@hd.run
def assoc_search(class_hvs, hv_matrix_tst):
    ret = hd.search(class_hvs, hv_matrix_tst)
    return ret

In [ ]:
with hd.utils.timing("Testing with class model\n"):
    search_results = assoc_search(class_hvs, hv_matrix_tst)

In [ ]:
search_results.to_numpy()

In [ ]:
validate(labels_tst, search_results.to_numpy())
